In [1]:
import requests
import re
import pandas as pd
import random
from string import ascii_lowercase
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [2]:
# Download NLTK stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to clean text: remove punctuation, stop words, and non-textual elements
def clean_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Split into words
    words = text.split()
    # Remove stop words and non-textual elements
    cleaned_words = [word for word in words if word.lower() not in stop_words]
    # strip underscores
    cleaned_words = [word.strip('_') for word in cleaned_words]
    # remove numbers
    cleaned_words = [word for word in cleaned_words if not word.isnumeric()]
    return ' '.join(cleaned_words)

def find_start_end(text):
    # Find the start and end of the main text
    start_pattern = r"\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .+ \*\*\*"
    end_pattern = r"\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .+ \*\*\*"

    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)

    start_idx = start_match.end() if start_match else 0
    end_idx = end_match.start() if end_match else len(text)

    return text[start_idx:end_idx]

def process_book(url, label):
    # Download the book text from the URL
    response = requests.get(url)
    response.encoding = 'utf-8'
    text = response.text

    # Extract the main text between start and end markers
    main_text = find_start_end(text)

    # Clean the main text
    cleaned_text = clean_text(main_text)

    # Extract words from the cleaned text
    words = cleaned_text.split()

    # Split words into partitions of 100 and take 200 random partitions
    partitions = [words[i:i + 100] for i in range(0, len(words), 100)]
    random_partitions = random.sample(partitions, min(200, len(partitions)))

    return [(label, ' '.join(partition)) for partition in random_partitions]

# Updated list of Gutenberg book URLs (Same as before, no change needed here)
book_urls = [
    'https://www.gutenberg.org/files/28054/28054-0.txt',  # The Brothers Karamazov
    'https://www.gutenberg.org/files/1998/1998-0.txt',    # Thus Spoke Zarathustra
    'https://www.gutenberg.org/files/2554/2554-0.txt',    # Crime and Punishment
    'https://www.gutenberg.org/files/7849/7849-0.txt',    # The Trial
    'https://www.gutenberg.org/files/600/600-0.txt',      # Notes from the Underground
    'https://www.gutenberg.org/files/205/205-0.txt'       # Walden
]

book_authors = ["Fyodor Dostoevsky", "Friedrich Nietzsche", "Fyodor Dostoevsky", "Franz Kafka",  "Fyodor Dostoevsky", "Henry David Thoreau"]


# Process all books
all_partitions = []

for url, label in zip(book_urls, book_authors):
    book_partitions = process_book(url, label)
    all_partitions.extend(book_partitions)

# Convert to DataFrame
partition_df = pd.DataFrame(all_partitions, columns=['Label', 'Words'])

# Serialize DataFrame to CSV
partition_df.to_csv('book_partitions_cleaned.csv', index=False)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier

In [26]:
model_objs = [
    OneVsRestClassifier(SVC(kernel="linear")),
    OneVsRestClassifier(SVC(kernel="rbf")),
    OneVsRestClassifier(SVC(kernel="poly")),
    RandomForestClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    OneVsOneClassifier(SGDClassifier()),
    DecisionTreeClassifier(),
    AdaBoostClassifier(),
    XGBClassifier(random_state=69)
    ]


model_names = [
    "Linear SVC",
    "Gaussian SVC",
    "Polynomial SVC",
    "RandomForestClassifier",
    "Naive Bayes",
    "KNeighborsClassifier",
    "SGDClassifier",
    "DecisionTreeClassifier",
    "AdaBoostClassifier",
    "XGBClassifier",
]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

kf = KFold(n_splits=10, shuffle=True, random_state=69)
onehot_encoder = OneHotEncoder(sparse=False, drop='first')

for m_obj, m_name in zip(model_objs, model_names):
  acc_sum = 0
  precision_sum = 0
  recall_sum = 0
  f1_sum = 0
  roc_auc_sum = 0
  for train_index,test_index in kf.split(partition_df[:600]):
    train_data = partition_df.iloc[train_index]
    test_data = partition_df.iloc[test_index]
    X_train = train_data.drop(['Label'], axis=1)
    y_train = train_data['Label']
    X_test = test_data.drop(['Label'], axis=1)
    y_test = test_data['Label']

    # One-hot encoding for the label
    y_train_onehot = onehot_encoder.fit_transform(train_data[['Label']])
    y_test_onehot = onehot_encoder.transform(test_data[['Label']])

    y_train_df = pd.DataFrame(y_train_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))
    y_test_df = pd.DataFrame(y_test_onehot, columns=onehot_encoder.get_feature_names_out(['Label']))

    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['Words'])
    X_test_tfidf = tfidf_vectorizer.transform(X_test['Words'])
    X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

    m_obj.fit(X_train_tfidf, y_train_df),
    y_pred = m_obj.predict(X_test_tfidf)
    acc = accuracy_score(y_pred, y_test_df)
    precision = precision_score(y_pred, y_test_df, average='macro')
    recall = recall_score(y_pred, y_test_df, average='macro')
    f1 = f1_score(y_pred, y_test_df, average='macro')
    roc_auc = roc_auc_score(y_test_df, y_pred, average='macro')

    acc_sum+=acc
    precision_sum += precision
    recall_sum += recall
    f1_sum += f1
    roc_auc_sum += roc_auc

  print("MODEL: {}".format(m_name))
  print("Average Accuracy: {}".format(acc_sum/10))
  print("Average Precision: {}".format(precision_sum/10))
  print("Average Recall: {}".format(recall_sum/10))
  print("Average F1: {}".format(f1_sum/10))
  print("Average ROC AUC: {}".format(roc_auc_sum/10))
  print()

MODEL: Linear SVC
Average Accuracy: 0.9816666666666667
Average Precision: 0.9781842425933173
Average Recall: 0.9807565258710322
Average F1: 0.9791160645503163
Average ROC AUC: 0.9781842425933173

MODEL: Gaussian SVC
Average Accuracy: 0.9716666666666667
Average Precision: 0.96080553054393
Average Recall: 0.9752759384616058
Average F1: 0.9668597847396516
Average ROC AUC: 0.96080553054393

